In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import time
import re

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=16)
client = Client(cluster)
client.dashboard_link

'http://127.0.0.1:8787/status'

2025-03-11 13:38:42,518 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:62663 (pid=25960) exceeded 95% memory budget. Restarting...
2025-03-11 13:38:43,934 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:62663' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('getitem-1861ac82fee65c3e8f8becb5ab641923', 3, 0), 'finalize-1e4898cd-39fe-40ca-86ce-23a042adf3ed', ('getitem-1861ac82fee65c3e8f8becb5ab641923', 16, 0), 'finalize-5f883999-61bc-4ecf-99ef-df2818888341', 'finalize-7b78f086-3d5f-4877-8ce9-89dbfff47b47', ('getitem-1861ac82fee65c3e8f8becb5ab641923', 40, 0), 'finalize-a4dcd0f2-a1e9-4148-8651-08addc47c975', 'finalize-c52d9e3b-73e7-4a1b-a5d5-27cca28df16d', 'finalize-6241984a-ae01-4b25-ba1e-e920802302b0', 'finalize-182c278a-8c54-4f08-8605-bd282d9649fa', 'finalize-16a4039c-ced0-4c63-ab65-222fa4ae05f1', 'finalize-35afcd37-5df5-4a7e-bfe2-c9966da57709', 'finalize-9af8a516-8b36-41c1-8d00-84d710a2e59c', 'finalize-96fc9f

In [3]:
import os 
import sys 
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from stream_cat_config import (
    LYR_DIR,
    MASK_DIR_RP100,
    MASK_DIR_SLP10,
    MASK_DIR_SLP20,
    ACCUM_DIR,
    NHD_DIR,
    OUT_DIR,
    PCT_FULL_FILE,
    PCT_FULL_FILE_RP100
)

from StreamCat_functions import (
    PointInPoly,
    createCatStats,
    mask_points
)

C:\Users\thudso02\AppData\Roaming\Python\Python312\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [4]:
ctl = pd.read_csv('../ControlTable_StreamCat.csv')
inter_vpu = pd.read_csv("../InterVPU.csv")
INPUTS = np.load(ACCUM_DIR +"/vpu_inputs.npy", allow_pickle=True).item()

already_processed = []

In [5]:
print(INPUTS)
print(len(INPUTS))
print(len(ctl.loc[ctl['run'] == 1]))

OrderedDict({'06': 'MS', '05': 'MS', '10U': 'MS', '10L': 'MS', '07': 'MS', '11': 'MS', '14': 'CO', '01': 'NE', '17': 'PN', '16': 'GB', '15': 'CO', '13': 'RG', '12': 'TX', '09': 'SR', '02': 'MA', '08': 'MS', '04': 'GL', '03W': 'SA', '03S': 'SA', '03N': 'SA', '18': 'CA'})
21
62


In [7]:
for _, row in ctl.query("run == 1").iterrows():

    apm = "" if row.AppendMetric == "none" else row.AppendMetric
    if row.use_mask == 1:
        mask_dir = MASK_DIR_RP100
    elif row.use_mask == 2:
        mask_dir = MASK_DIR_SLP10
    elif row.use_mask == 3:
        mask_dir = MASK_DIR_SLP20
    else:
        mask_dir = ""
    landscape_layer_year = re.findall(r'\d+', row.LandscapeLayer)
    actual_layer_name = f"Annual_NLCD_LndCov_{landscape_layer_year[0]}_CU_C1V0.tif"
    layer = (
        actual_layer_name
        if "/" in row.LandscapeLayer or "\\" in row.LandscapeLayer
        else (f"{LYR_DIR}/{actual_layer_name}")
    )  # use abspath
    print(layer)
    if isinstance(row.summaryfield, str):
        summary = row.summaryfield.split(";")
    else:
        summary = None
    if row.accum_type == "Point":
        # Load in point geopandas table and Pct_Full table
        # TODO: script to create this PCT_FULL_FILE
        pct_full = pd.read_csv(
            PCT_FULL_FILE if row.use_mask == 0 else PCT_FULL_FILE_RP100
        )
        points = gpd.read_file(layer)
        if mask_dir:
            points = mask_points(points, mask_dir, INPUTS)
    # File string to store InterVPUs needed for adjustments
    Connector = f"{OUT_DIR}/{row.FullTableName}_connectors.csv"
    print(
        f"Acquiring `{row.FullTableName}` catchment statistics...",
        end="",
        flush=True,
    )
    start_time = time.time()
    for zone, hydroregion in INPUTS.items():
        zone_start_time = time.time()
        if not os.path.exists(f"{OUT_DIR}/{row.FullTableName}_{zone}.csv"):
            print(zone, end=", ", flush=True)
            pre = f"{NHD_DIR}/NHDPlus{hydroregion}/NHDPlus{zone}"
            if not row.accum_type == "Point":
                izd = (
                    f"{mask_dir}/{zone}.tif"
                    if mask_dir
                    else f"{pre}/NHDPlusCatchment/cat"
                )
                cat = createCatStats(
                    row.accum_type,
                    layer,
                    izd,
                    OUT_DIR,
                    zone,
                    row.by_RPU,
                    mask_dir,
                    NHD_DIR,
                    hydroregion,
                    apm,
                    use_dask=True
                )
            if row.accum_type == "Point":
                izd = f"{pre}/NHDPlusCatchment/Catchment.shp"
                cat = PointInPoly(
                    points, zone, izd, pct_full, mask_dir, apm, summary, use_dask=True
                )
            cat.to_csv(f"{OUT_DIR}/{row.FullTableName}_{zone}.csv", index=False)
            zone_end_time = time.time()
            print(f"Time to finish processing stats for zone {zone} / region {hydroregion} {(zone_end_time - zone_start_time) / 60} minutes")
    end_time = time.time()
    print(f"Processed {len(INPUTS)} regions for one metric {row.FullTableName} in {end_time-start_time} seconds using a dask and 8 workers")
    print("done!")

O:/PRIV/CPHEA/PESD/COR/CORFiles/Geospatial_Library_Projects/StreamCat/LandscapeRasters/QAComplete/Annual_NLCD_LndCov_1987_CU_C1V0.tif
Acquiring `N_Human_Waste_1987` catchment statistics...09, 

C:\Users\thudso02\AppData\Roaming\Python\Python312\site-packages\distributed\client.py:3362: UserWarning: Sending large graph of size 1.38 GiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Time to finish processing stats for zone 09 / region SR 9.661166687806448 minutes
02, 

C:\Users\thudso02\AppData\Roaming\Python\Python312\site-packages\distributed\client.py:3362: UserWarning: Sending large graph of size 1.02 GiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Time to finish processing stats for zone 02 / region MA 14.2870729525884 minutes
08, 

...

Time to finish processing stats for zone 05 / region MS 11.17708779970805 minutes
10U, 

C:\Users\thudso02\AppData\Roaming\Python\Python312\site-packages\distributed\client.py:3362: UserWarning: Sending large graph of size 1.69 GiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Time to finish processing stats for zone 10U / region MS 36.679597775141396 minutes
10L, 

C:\Users\thudso02\AppData\Roaming\Python\Python312\site-packages\distributed\client.py:3362: UserWarning: Sending large graph of size 1.46 GiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


KeyboardInterrupt: 